# Decision Tree Classifier & Regressor

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

import shap

import warnings
warnings.filterwarnings('ignore')

In [15]:
data = pd.read_csv('opencnil-violationsdcpnotifiees-20241231.csv', sep = ';')
data.head()

,Date de réception de la notification,Secteur d'activité de l'organisme concerné,Natures de la violation,Nombre de personnes impactées,Typologies des données impactées,Données sensibles,Origines de l'incident,Causes de l'incident,Information des personnes
0,2024-12,"Activités spécialisées, scientifiques et techn...",Perte de la confidentialité,Entre 301 et 5000 personnes,"Etat civil (ex : nom, sexe, date de naissance,...",NaN,"Piratage, logiciel malveillant (par exemple ra...",Acte externe malveillant,"Oui, les personnes ont été informées"
1,2024-12,Administration publique,Perte de la confidentialité,Entre 6 et 50 personnes,NaN,Oui,"Equipement perdu ou volé,Papier perdu, volé ou...",Acte externe malveillant,Non déterminé pour le moment
2,2024-12,Information et communication,Perte de la disponibilité,Entre 51 et 300 personnes,"Etat civil (ex : nom, sexe, date de naissance,...",NaN,"Piratage, logiciel malveillant (par exemple ra...",Acte externe malveillant,"Non, mais elles le seront"
3,2024-12,Autres activités de services,Perte de la confidentialité,Entre 51 et 300 personnes,Coordonnées (ex : adresse postale ou électroni...,NaN,Données personnelles envoyées à un mauvais des...,Acte interne accidentel,"Non, mais elles le seront"
4,2024-12,"Activités spécialisées, scientifiques et techn...","Perte de la confidentialité,Perte de l'intégri...",Entre 6 et 50 personnes,Coordonnées (ex : adresse postale ou électroni...,NaN,"Piratage, logiciel malveillant (par exemple ra...",Acte externe malveillant,"Oui, les personnes ont été informées"


## Data analysis

- `Date de réception de la notification` \- Ordinal 
- `Secteur d'activité de l'organisme concerné` \- Nominal
- `Natures de la violation` \- Nominal 
- `Nombre de personnes impactées` \- Ordinal
- `Typologies des données impactées` \- Nominal
- `Données sensibles` \- Nominal value to predict
- `Origines de l'incident` \- Nominal 
- `Causes de l'incident` \- Nominal
- `Information des personnes` \- Nominal

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25697 entries, 0 to 25696
Data columns (total 9 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Date de réception de la notification        25697 non-null  object
 1   Secteur d'activité de l'organisme concerné  25697 non-null  object
 2   Natures de la violation                     25697 non-null  object
 3   Nombre de personnes impactées               25697 non-null  object
 4   Typologies des données impactées            25545 non-null  object
 5   Données sensibles                           4913 non-null   object
 6   Origines de l'incident                      25697 non-null  object
 7   Causes de l'incident                        25697 non-null  object
 8   Information des personnes                   25697 non-null  object
dtypes: object(9)
memory usage: 1.8+ MB


In [17]:
# Checking for missing values
data.isnull().sum()

Date de réception de la notification              0
Secteur d'activité de l'organisme concerné        0
Natures de la violation                           0
Nombre de personnes impactées                     0
Typologies des données impactées                152
Données sensibles                             20784
Origines de l'incident                            0
Causes de l'incident                              0
Information des personnes                         0
dtype: int64

In [18]:
# get unique values in each column
for col in data.columns:
    print(f"{col}: {data[col].nunique()} unique values")
    print(data[col].unique())
    print("\n")

Date de réception de la notification : 80 unique values
['2024-12' '2024-11' '2024-10' '2024-09' '2024-08' '2024-07' '2024-06'
 '2024-05' '2024-04' '2024-03' '2024-02' '2024-01' '2023-12' '2023-11'
 '2023-10' '2023-09' '2023-08' '2023-07' '2023-06' '2023-05' '2023-04'
 '2023-03' '2023-02' '2023-01' '2022-12' '2022-11' '2022-10' '2022-09'
 '2022-08' '2022-07' '2022-06' '2022-05' '2022-04' '2022-03' '2022-02'
 '2022-01' '2021-12' '2021-11' '2021-10' '2021-09' '2021-08' '2021-07'
 '2021-06' '2021-05' '2021-04' '2021-03' '2021-02' '2021-01' '2020-12'
 '2020-11' '2020-10' '2020-09' '2020-08' '2020-07' '2020-06' '2020-05'
 '2020-04' '2020-03' '2020-02' '2020-01' '2019-12' '2019-11' '2019-10'
 '2019-09' '2019-08' '2019-07' '2019-06' '2019-05' '2019-04' '2019-03'
 '2019-02' '2019-01' '2018-12' '2018-11' '2018-10' '2018-09' '2018-08'
 '2018-07' '2018-06' '2018-05']


Secteur d'activité de l'organisme concerné: 22 unique values
['Activités spécialisées, scientifiques et techniques'
 'Administrat

In [19]:
# get unique value counts in each column
for col in data.columns:
    print(f"{col}:")
    print(data[col].value_counts())
    print("\n")

Date de réception de la notification :
Date de réception de la notification 
2022-02    1147
2021-05     858
2024-02     825
2021-07     788
2021-12     687
           ... 
2019-08     129
2018-12     128
2018-08     127
2018-09     103
2018-05      17
Name: count, Length: 80, dtype: int64


Secteur d'activité de l'organisme concerné:
Secteur d'activité de l'organisme concerné
Administration publique                                                                                                                                4564
Activités spécialisées, scientifiques et techniques                                                                                                    3412
Activités financières et d''assurance                                                                                                                  3085
Santé humaine et action sociale                                                                                                                        2

In [20]:
# get number of unique values per column
data.nunique()

Date de réception de la notification           80
Secteur d'activité de l'organisme concerné     22
Natures de la violation                         7
Nombre de personnes impactées                   5
Typologies des données impactées              381
Données sensibles                               1
Origines de l'incident                        126
Causes de l'incident                           39
Information des personnes                       4
dtype: int64

In [30]:
# get counts of multi-label Natures de la violation:
nature_counts = data['Natures de la violation'].value_counts()
print(nature_counts)

# Split the multi-label column into separate binary columns
nature_dummies = data['Natures de la violation'].str.get_dummies(sep=',')
nature_dummies.head()
# shows counts of each nature
nature_dummies.sum().sort_values(ascending=False).to_string().split('\n')

Natures de la violation
Perte de la confidentialité                                                   17594
Perte de la confidentialité,Perte de la disponibilité                          2296
Perte de la disponibilité                                                      2060
Perte de la confidentialité,Perte de l'intégrité,Perte de la disponibilité     1695
Perte de la confidentialité,Perte de l'intégrité                               1131
Perte de l'intégrité,Perte de la disponibilité                                  578
Perte de l'intégrité                                                            343
Name: count, dtype: int64


['Perte de la confidentialité    22716',
 'Perte de la disponibilité       6629',
 "Perte de l'intégrité            3747"]

## Decision on the question

Question

> Given an incident's cause and origin can we predict the type of violation?

Will train 3 binary classification models:

- **Model 1**: Predicts "Does this incident involve confidentiality loss?" (22,716 yes vs ~2,900 no)
- **Model 2**: Predicts "Does this incident involve availability loss?" (6,629 yes vs ~19,000 no)
- **Model 3**: Predicts "Does this incident involve integrity loss?" (3,747 yes vs ~21,900 no)



### Preprocessing features (feature engineering...?)

**Input complexity**: 126 for Origins and 39 for causes - both are multi-label data.

Origins is possibly unmanageable and Cause appears manageable (assuming preprocessing doesn't explode this number)

**Output complexity**: 3 binary classification models rather than 7 ((2^3)-1) classification labels on one model

**Data pattern**: Clear categorial problem


In [34]:
# Split the multi-label column into separate binary columns
cause_dummies = data["Origines de l\'incident"].str.get_dummies(sep=',')
cause_dummies.head()
# shows counts of each nature
cause_dummies.sum().sort_values(ascending=False).to_string().split('\n')

[' logiciel malveillant (par exemple rançongiciel) et/ou hameçonnage                                  13500',
 'Piratage                                                                                            13500',
 'Autre                                                                                                5070',
 'Données personnelles envoyées à un mauvais destinataire                                              2950',
 "Publication non volontaire d'informations                                                            2217",
 'Equipement perdu ou volé                                                                             1996',
 'Données de la mauvaise personne affichées sur le portail du client                                    731',
 'Papier perdu                                                                                          687',
 ' volé ou laissé accessible dans un endroit non sécurisé                                               687',
 'Informat

In [33]:
# Split the multi-label column into separate binary columns
cause_dummies = data["Causes de l\'incident"].str.get_dummies(sep=',')
cause_dummies.head()
# shows counts of each nature
cause_dummies.sum().sort_values(ascending=False).to_string().split('\n')

['Acte externe malveillant    15148',
 'Acte interne accidentel      5244',
 'Inconnu                      2196',
 'Autre                        1767',
 'Acte externe accidentel      1402',
 'Acte interne malveillant     1220']

### Update to preprocessing 

The features are a lot more tractable with 13 Origins and 6 causes


In [ ]:
# For causes - create 6 binary features
data['cause_external_malicious'] = data['Causes de l\'incident'].str.contains('Acte externe malveillant', na=False)
data['cause_internal_accidental'] = data['Causes de l\'incident'].str.contains('Acte interne accidentel', na=False)
data['cause_external_accidental'] = data['Causes de l\'incident'].str.contains('Acte externe accidentel', na=False)
data['cause_internal_malicious'] = data['Causes de l\'incident'].str.contains('Acte interne malveillant', na=False)
data['cause_unknown'] = data['Causes de l\'incident'].str.contains('Inconnu', na=False)
data['cause_other'] = data['Causes de l\'incident'].str.contains('Autre', na=False)

# Check the feature extraction worked
cause_features = [col for col in data.columns if col.startswith('cause_')]
# print cause features created in table format
cause_feature_counts = {feature: data[feature].sum() for feature in cause_features}
cause_feature_df = pd.DataFrame(list(cause_feature_counts.items()), columns=['Cause Feature', 'Count'])
cause_feature_df                                               


,Origin Feature,Count
0,origin_malware_phishing,13500
1,origin_misdirected_data,2950
2,origin_involuntary_publication,2217
3,origin_lost_stolen_equipment,1996
4,origin_lost_paper,687
5,origin_verbal_disclosure,224
6,origin_lost_mail,174
7,origin_improper_disposal,89
8,origin_wrong_person_portal,731
9,origin_other,5070


In [46]:


# For origins - create binary features for the main categories
data['origin_malware_phishing'] = data['Origines de l\'incident'].str.contains('logiciel malveillant.*hameçonnage|Piratage', na=False)
data['origin_misdirected_data'] = data['Origines de l\'incident'].str.contains('Données personnelles envoyées à un mauvais destinataire', na=False)
data['origin_involuntary_publication'] = data['Origines de l\'incident'].str.contains('Publication non volontaire d\'informations', na=False)
data['origin_lost_stolen_equipment'] = data['Origines de l\'incident'].str.contains('Equipement perdu ou volé', na=False)
data['origin_lost_paper'] = data['Origines de l\'incident'].str.contains('Papier perdu.*accessible.*non sécurisé', na=False)
data['origin_verbal_disclosure'] = data['Origines de l\'incident'].str.contains('Informations personnelles divulguées.*verbale', na=False)
data['origin_lost_mail'] = data['Origines de l\'incident'].str.contains('Courrier perdu.*retourné', na=False)
data['origin_improper_disposal'] = data['Origines de l\'incident'].str.contains('Mise au rebut.*sans.*sécurisé|sans destruction physique', na=False)
data['origin_wrong_person_portal'] = data['Origines de l\'incident'].str.contains('mauvaise personne.*portail', na=False)
data['origin_other'] = data['Origines de l\'incident'].str.contains('Autre', na=False)

# Check the feature extraction worked
origin_features = [col for col in data.columns if col.startswith('origin_')]

# print origin features created in table format
origin_feature_counts = {feature: data[feature].sum() for feature in origin_features}
origin_feature_df = pd.DataFrame(list(origin_feature_counts.items()), columns=['Origin Feature', 'Count'])
origin_feature_df

,Origin Feature,Count
0,origin_malware_phishing,13500
1,origin_misdirected_data,2950
2,origin_involuntary_publication,2217
3,origin_lost_stolen_equipment,1996
4,origin_lost_paper,687
5,origin_verbal_disclosure,224
6,origin_lost_mail,174
7,origin_improper_disposal,89
8,origin_wrong_person_portal,731
9,origin_other,5070


## Modelling

How is the data balanced? 

- **Availability**: 6,629 yes vs ~19,000 no (roughly 1:3 ratio - most balanced)
- **Integrity**: 3,747 yes vs ~21,900 no (roughly 1:6 ratio)
- **Confidentiality**: 22,716 yes vs ~2,900 no (roughly 8:1 ratio - most imbalanced)
